In [2]:
# Imports

import pandas as pd
import numpy as ny

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# showing all dataframe columns:

pd.set_option('display.max_columns', None)

In [ ]:
# set matplotlib global settings eg. figsize:
plt.rcParams['figure.figsize'] = (8.0, 6.0)

In [3]:
# Read London Airbnb Data:
airbnb_ldn = pd.read_csv('data/LondonData.csv')

Option settings:

In [4]:
# First look into raw dataframe:
airbnb_ldn.head().T

0  \
Listing Title                                      Twin Ensuite with Shower at Colnbrook Lodge Gu...   
Property Type                                                             Private room in guesthouse   
Listing Type                                                                            private_room   
Created Date                                                                              2022-07-23   
Last Scraped Date                                                                         2022-07-27   
Country                                                                                           GB   
State                                                                                             UK   
City                                                                                       Colnbrook   
Zipcode                                                                                        SL3 0   
Currency Native                                                                                  GBP   
Number of Reviews                                                                                  0   
Bedrooms                                                                                           1   
Bathrooms                                                                                          1   
Max Guests                                                                                         2   
Airbnb Superhost                                                                                   f   
Cancellation Policy                                                                              NaN   
Cleaning Fee (USD)                                                                               0.0   
Cleaning Fee (Native)                                                                            0.0   
Extra People Fee (USD)                                                                           0.0   
Extra People Fee(Native)                                                                         0.0   
Check-in Time                                                                     3:00 PM - 11:00 PM   
Checkout Time                                                                               11:00 AM   
Minimum Stay                                                                                       1   
Latitude                                                                                    51.47957   
Longitude                                                                                   -0.50175   
Exact Location                                                                                     f   
Overall Rating                                                                                   NaN   
Airbnb Communication Rating                                                                      NaN   
Airbnb Accuracy Rating                                                                           NaN   
Airbnb Cleanliness Rating                                                                        NaN   
Airbnb Checkin Rating                                                                            NaN   
Airbnb Location Rating                                                                           NaN   
Airbnb Value Rating                                                                              NaN   
Amenities                                          ['Free parking on premises', 'Wifi', 'Breakfas...   
picture_url                                        https://a0.muscache.com/im/pictures/prohost-ap...   
License                                                                                          NaN   
Airbnb Property ID                                                                          53931060   
Airbnb Host ID                                                                             436870537   
Host Listing Count                         

In [5]:
# checking dimensions of dataframe:
airbnb_ldn.shape

(43580, 54)

The dataframe consists of 54 columns and 43580 rows.

In [6]:
# More thorough check of data included in dataset:
airbnb_ldn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43580 entries, 0 to 43579
Data columns (total 54 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Listing Title                                      43576 non-null  object 
 1   Property Type                                      43580 non-null  object 
 2   Listing Type                                       43580 non-null  object 
 3   Created Date                                       43580 non-null  object 
 4   Last Scraped Date                                  43580 non-null  object 
 5   Country                                            43580 non-null  object 
 6   State                                              43580 non-null  object 
 7   City                                               43580 non-null  object 
 8   Zipcode                                            43580 non-null  object 
 9   Curren

Some more indepth analysis of the various columns will be conducted. 

Any columns that are not necessary for the modelling process shall be dropped. 
- Columns that are semantically the same will be dropped.
- Columns that do not serve any relevant purpose and don't add additional value to the future insights will be dropped.

To begin, columns that can be deemed uneccessary will be dropped:

- The two columns 'country' and 'state' can be dropped. As this is the London dataset, all the pieces of data on these columns will be the same and won't   serve any values. 
- There are numerous columns that have some monetary information. There are duplicates, with columns being in both native currency (GBP) and USD. When models are created, these columns will be colinear (they are the same information just scaled). For that reason, the USD columns will be dropped (they could be reincluded at a later date, if comparison are to be made between various cities). The columns that have these currency duplicates are: 'Cleaning fee', 'Extra people fee', 'Average daily rate' and 'Annual revenue LTM'. These columns will all be dropped.
- Two other columns which don't add to the insights are the 'Date Created' and the 'Last scraped' column.
The relevant timeframe values are all based on the 'Last Twelve Months' [LTM]. Where the timeframe is relevant, the LTM detail has been listed in the column title. Knowledge of when the column was created and when data was last scraped from it, does not aid in any form of comparative insight.
- License column, is made up entirely of null values. Can be dropped.
- The instant bookable column also contains many null values, and isn't a particularly valuable independent value, this column will be dropped. Instantbook availalbe is not very useful, will be dropped too.
- Drop 'currency native' column, it does not add any value (all data is known to be in GBP).
- Drop property ID column (it does not add any particular value to the dataframe and potential analysis).
- Drop Listing URL, not relevant information.
- Picture url can be dropped.



A quick check will be conducted, to ensure these columns do not have any greater value than initially assumed:

In [9]:
# Dropping descrbied columns:
airbnb_ldn.drop(columns =['Country', 'State', 'Cleaning Fee (USD)', 'Extra People Fee (USD)', 
                          'Average Daily Rate (USD)', 'Annual Revenue LTM (USD)','Created Date', 
                          'Last Scraped Date', 'License', 'instant_bookable', 'Currency Native',
                          'Airbnb Property ID', 'Listing URL', 'Exact Location', 'Instantbook Enabled',
                          'picture_url'], inplace=True )

KeyError: "['Country', 'State', 'Cleaning Fee (USD)', 'Extra People Fee (USD)', 'Average Daily Rate (USD)', 'Annual Revenue LTM (USD)', 'Created Date', 'Last Scraped Date', 'License', 'instant_bookable', 'Currency Native', 'Airbnb Property ID', 'Listing URL', 'Exact Location', 'Instantbook Enabled', 'picture_url'] not found in axis"

In [8]:
airbnb_ldn.shape

(43580, 38)

There are now 38 columns, indicating the 16 listed columns deemed unecessary have now been dropped. 

Some of the row information will be checked, to determine where there are present null values and what their patterns/relevance could be:

In [10]:
sns.heatmap(airbnb_ldn.isnull(), yticklabels=False, cbar=False)

<Axes: >

There are many lines of data that have null values for all of the different 'rating' columns. For the purpose of this model, the rating component will be essential to the ultimate output. Hence, columns that do not have this relevant information will not be useful for any form of insights. All columns that have null values for all rating rows will be dropped:

In [15]:
# return columns that have null values for all rating rows:

airbnb_ldn[airbnb_ldn[['Overall Rating', 
            'Airbnb Communication Rating', 
            'Airbnb Accuracy Rating', 
            'Airbnb Cleanliness Rating',
            'Airbnb Checkin Rating',
            'Airbnb Location Rating',
            'Airbnb Value Rating']].isnull().any(axis=1)]

Listing Title  \
64     Double room - fantastic location close to Heat...   
86                      Spacious Double Room in Hounslow   
116                            The Hideaway at Valentina   
142    Amazing 2bed house with garden and street parking   
209                        Single En-suite Room from £35   
...                                                  ...   
43343             Lovely shared house , ideally located.   
43344      great location with easy access to everything   
43385                           Central London Apartment   
43414  LOVELY DOUBLE ROOM IN LONDON RM10,30minTO CENTRAL   
43471      15 Mins from Stratford| Minimalist New Double   

                             Property Type  Listing Type            City  \
64     Private room in condominium (condo)  private_room  Greater London   
86                   Private room in house  private_room  Greater London   
116              Private room in houseboat  private_room         Hampton   
142                Entire residential home   entire_home  Greater London   
209              Private room in apartment  private_room  Greater London   
...                                    ...           ...             ...   
43343            Private room in townhouse  private_room  Greater London   
43344            Private room in townhouse  private_room  Greater London   
43385                   Entire rental unit   entire_home          London   
43414                Private room in house  private_room  Greater London   
43471            Private room in apartment  private_room        Dagenham   

      Zipcode  Number of Reviews Bedrooms  Bathrooms  Max Guests  \
64     TW13 5                  0        1          1           2   
86      TW4 7                  4        1          1           2   
116    TW12 2                 11        1          1           2   
142    TW11 8                  0        2          2           5   
209     KT1 2                  3        1          1           2   
...       ...                ...      ...        ...         ...   
43343   IG6 1                  0        1          3           2   
43344   IG6 1                  0        1          1           3   
43385  RM13 8                  0        1          1           3   
43414  RM10 7                  0        1          1           1   
43471   RM6 6                  0        1          1           2   

      Airbnb Superhost  ...  \
64                   f  ...   
86                   t  ...   
116                  f  ...   
142                  t  ...   
209                  f  ...   
...                ...  ...   
43343                f  ...   
43344                f  ...   
43385                f  ...   
43414                f  ...   
43471                f  ...   

                                          guest_controls  Pets Allowed  \
64     {"allows_children": true, "allows_infants": fa...         False   
86     {"allows_children": true, "allows_infants": tr...         False   
116    {"allows_children": false, "allows_infants": f...         False   
142    {"allows_children": false, "allows_infants": f...         False   
209    {"allows_children": true, "allows_infants": fa...         False   
...                                                  ...           ...   
43343  {"allows_children": false, "allows_infants": f...         False   
43344  {"allows_children": false, "allows_infants": f...         False   
43385  {"allows_children": true, "allows_infants": fa...         False   
43414  {"allows_children": false, "allows_infants": f...         False   
43471  {"allows_children": true, "allows_infants": tr...         False   

       Count Available Days LTM Count Blocked Days LTM  \
64                            3                    362   
86                            4                    361   
116                           0                    365   
142                           0                    365   
209                           0         

There are rows that have null values just on certain columns of the different column ratings. For now, only those columns that have null values for the 'Overall Rating' column will be dropped:

In [16]:
# drop all rows with null values in 'Overall Rating' column:
airbnb_ldn.dropna(subset=['Overall Rating'], inplace=True)

Check for duplicate rows:

In [17]:
airbnb_ldn.duplicated().sum()

0

There are a vast quantity of properties within this dataset, covering a range of different areas. The dataset contains a large quantity of properties that are on the outer edges of London. For the purpose of this project, only properties within the London postal districts will be considered, these are: W, SW, SE, E, N, NW, WC and EC. All rows that do not have a postcode that starts with one of these 8 letter combinations will be dropped.

In [18]:
# create a dataframe, containing only those rows that have the relevant postcodes:
ldn_pd_airbnb = airbnb_ldn[airbnb_ldn['Zipcode'].str.slice(0,1).str.contains('W') |
            airbnb_ldn['Zipcode'].str.slice(0,1).str.contains('E') |
            airbnb_ldn['Zipcode'].str.slice(0,2).str.contains('SE') |
            airbnb_ldn['Zipcode'].str.slice(0,2).str.contains('SW') |
            airbnb_ldn['Zipcode'].str.slice(0,2).str.contains('NE') |
            airbnb_ldn['Zipcode'].str.slice(0,1).str.contains('N') |
            airbnb_ldn['Zipcode'].str.slice(0,2).str.contains('NW') |
            airbnb_ldn['Zipcode'].str.slice(0,2).str.contains('WC') |
            airbnb_ldn['Zipcode'].str.slice(0,3).str.contains('EC') 
            ]

Check the shape of the London postcode dataframe:

In [19]:
ldn_pd_airbnb.shape

(32686, 38)

In [20]:
airbnb_ldn_final = ldn_pd_airbnb.reset_index(drop=True).copy()

This dataframe will be saved as a CSV:

In [ ]:
airbnb_ldn_final.to_csv('airbnb_ldn_final.csv')